In [ ]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar

--2025-05-13 04:27:04--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 54.231.227.113, 16.182.100.225, 52.217.160.57, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|54.231.227.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253381120 (3.0G) [application/x-tar]
Saving to: ‘abo-images-small.tar’

abo-images-small.ta 100%[===================>]   3.03G  14.0MB/s    in 3m 39s  

2025-05-13 04:30:44 (14.1 MB/s) - ‘abo-images-small.tar’ saved [3253381120/3253381120]



In [ ]:
!tar -xf abo-images-small.tar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers peft accelerate datasets bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00


In [ ]:
import os
import torch
from torch.utils.data import IterableDataset, DataLoader
from transformers import BlipProcessor, BlipForQuestionAnswering, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
from PIL import Image
import pandas as pd
from transformers import Trainer, TrainingArguments

In [ ]:
# ========= CONFIG =========
listing = "4"
csv_path = '/content/drive/MyDrive/images/VQA_dataset_train/listings_4_VQA_train.csv'  # CSV must contain: image_path, question, one_word_answer
#get length of csv file
# csv_path = f"/content/drive/MyDrive/images/VQA-dataset-train/listings_{listing}_VQA_train.csv"
f = open(csv_path, "r")
lines = f.readlines()
f.close()

model_name = "Salesforce/blip-vqa-base"
output_dir = f"/content/drive/MyDrive/images/lora_on_listing_{listing}"
images_root = "/content/images/small"
batch_size = 16
max_steps = (len(lines)-1)//batch_size
print(f"Steps per epoch: {max_steps}")
checkpoint_path = '/content/drive/MyDrive/images/lora_on_listing_0/'
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
del lines

Steps per epoch: 9693


In [ ]:

class VQALazyIterableDataset(IterableDataset):
    def __init__(self, dataset_stream, processor):
        self.dataset_stream = dataset_stream
        self.processor = processor
        self.images_root = "/content/images/small"

    def preprocess(self, example):
        try:

            image = Image.open(f"{self.images_root}/{example['image_path']}").convert("RGB")

            inputs = self.processor(
                image,
                example["question"],
                return_tensors="pt",
                max_length=50,
                truncation=True,
                padding="max_length"
            )

            labels = self.processor.tokenizer(
                text=example["answer"],
                return_tensors="pt",
                max_length=20,
                truncation=True,
                padding="max_length"
            ).input_ids

            return {
                "pixel_values": inputs["pixel_values"].squeeze(0),
                "input_ids": inputs["input_ids"].squeeze(0),
                "attention_mask": inputs["attention_mask"].squeeze(0),
                "labels": labels.squeeze(0)
            }
        except Exception as e:
            print(f"Skipping example due to error: {e}")
            return None

    def __iter__(self):
        for example in self.dataset_stream:
            processed = self.preprocess(example)
            if processed:
                yield processed


In [ ]:
# Stream CSV dataset
# streamed_dataset = load_dataset("csv", data_files=csv_path, split="train", streaming=True)
df = pd.read_csv(csv_path)
streamed_dataset = Dataset.from_pandas(df).to_iterable_dataset()

streamed_dataset = streamed_dataset.filter(
    lambda x: x["image_path"] and x["question"] and x["answer"]
)

processor = BlipProcessor.from_pretrained(model_name)

# Wrap in PyTorch Dataset
torch_dataset = VQALazyIterableDataset(streamed_dataset, processor)

# Create DataLoader
# dataloader = DataLoader(torch_dataset, batch_size=batch_size, num_workers=2)


In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig, TaskType
from functools import partial
base_model = BlipForQuestionAnswering.from_pretrained(model_name, device_map="auto")
# PEFT Config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "key", "value", "fc1", "fc2"],
    task_type=TaskType.QUESTION_ANS
)
if checkpoint_path is not None:
    print(f"Loading checkpoint from {checkpoint_path}")
    config = PeftConfig.from_pretrained(checkpoint_path)
    model = PeftModel.from_pretrained(base_model, checkpoint_path)

else:
    print("Training from scratch")
    model = get_peft_model(base_model, peft_config)
def new_forward(self, *args, **kwargs):
    # Exclude 'inputs_embeds' from kwargs if present
    kwargs.pop("inputs_embeds", None)
    return self.base_model.forward(*args, **kwargs)

# Apply the modified forward method to your model
model.forward = partial(new_forward, model)

Loading checkpoint from /content/drive/MyDrive/images/lora_on_listing_0/


In [ ]:

args = TrainingArguments(
      output_dir=output_dir,
      per_device_train_batch_size=batch_size,
      gradient_accumulation_steps= 1,
      warmup_ratio=0.1,
      save_strategy="steps",
      logging_steps=1000,
      learning_rate=2e-5,
      fp16=True,
      max_steps=max_steps,
      report_to=[],  # ✅ disables wandb, tensorboard, etc.
      save_total_limit=2,
      save_steps=1000,
  )
trainer = Trainer(
      model=model,
      args=args,
      train_dataset=torch_dataset,
      # data_collator=default_data_collator
  )
model.print_trainable_parameters()
trainer.train()

  # Save model and processor
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 0 || all params: 387,179,324 || trainable%: 0.0000


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")


Model saved to /content/drive/MyDrive/images/lora_on_listing_0
